# Data Preprocessing & EDA

## Attaching Resources

### Importing Necessary Libraries

In [121]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as pe
import statsmodels.api as sm
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import IsolationForest

import warnings

warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = (15, 8)
plt.rcParams['figure.dpi'] = 350
sns.set_style(style='darkgrid')
plt.tight_layout()
%matplotlib inline

### Adding Dataset

In [3]:
book_raw = pd.read_csv('https://raw.githubusercontent.com/kunal-mallick/Book_Recommendation/main/resources/Books.csv', encoding='latin-1')

book_raw.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [189]:
book_raw.shape

(271360, 8)

In [5]:
ratings_raw = pd.read_csv('https://raw.githubusercontent.com/kunal-mallick/Book_Recommendation/main/resources/Ratings.csv', encoding='latin-1')

ratings_raw.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [188]:
ratings_raw.shape

(1149780, 3)

In [6]:
users_raw = pd.read_csv('https://raw.githubusercontent.com/kunal-mallick/Book_Recommendation/main/resources/Users.csv', encoding='latin-1')

users_raw.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


## Performing EDA In Dataset

### Performing Data Preprocessing

#### Applying Descriptive Statistics

In [6]:
book_raw.describe()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
count,271360,271360,271359,271360,271358,271360,271360,271357
unique,271360,242135,102023,202,16807,271044,271044,271041
top,0195153448,Selected Poems,Agatha Christie,2002,Harlequin,http://images.amazon.com/images/P/185326119X.0...,http://images.amazon.com/images/P/185326119X.0...,http://images.amazon.com/images/P/225307649X.0...
freq,1,27,632,13903,7535,2,2,2


In [7]:
ratings_raw.describe()

,User-ID,Book-Rating
count,1.149780e+06,1.149780e+06
mean,1.403864e+05,2.866950e+00
std,8.056228e+04,3.854184e+00
min,2.000000e+00,0.000000e+00
25%,7.034500e+04,0.000000e+00
50%,1.410100e+05,0.000000e+00
75%,2.110280e+05,7.000000e+00
max,2.788540e+05,1.000000e+01


In [8]:
users_raw.describe()

,User-ID,Age
count,278858.00000,168096.000000
mean,139429.50000,34.751434
std,80499.51502,14.428097
min,1.00000,0.000000
25%,69715.25000,24.000000
50%,139429.50000,32.000000
75%,209143.75000,44.000000
max,278858.00000,244.000000


#### Performing Typecasting

In [9]:
book_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271359 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


In [10]:
ratings_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


In [11]:
users_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   User-ID   278858 non-null  int64  
 1   Location  278858 non-null  object 
 2   Age       168096 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB


#### Performing Missing Value Imputation

In [12]:
def missing_percentage(wp):

    m = wp.isna().sum()
    total = int(wp.shape[0])

    for i in range(len(wp.columns)):
        percentage =round((m[i]/total)*100)

        print(str(wp.columns[i]) + ' has ' + str(percentage) + ' % missing value(' + str(m[i]) + ')')

##### Missing Value Percentage

In [13]:
missing_percentage(book_raw)

ISBN has 0 % missing value(0)
Book-Title has 0 % missing value(0)
Book-Author has 0 % missing value(1)
Year-Of-Publication has 0 % missing value(0)
Publisher has 0 % missing value(2)
Image-URL-S has 0 % missing value(0)
Image-URL-M has 0 % missing value(0)
Image-URL-L has 0 % missing value(3)


In [14]:
missing_percentage(ratings_raw)

User-ID has 0 % missing value(0)
ISBN has 0 % missing value(0)
Book-Rating has 0 % missing value(0)


In [15]:
missing_percentage(users_raw)

User-ID has 0 % missing value(0)
Location has 0 % missing value(0)
Age has 40 % missing value(110762)


##### Removing Missing Value

##### Missing Value Percentage(After)

In [16]:
#missing_percentage(users_raw)

#### Finding Unique Values In Dataset

In [75]:
def uni(df):

    for i in range(len(df.columns)):
        print('\n All Unique Value in ' + str(df.columns[i]))
        print(np.sort(df[df.columns[i]].unique()))
        print('Total no of unique values ' +
              str(len(df[df.columns[i]].unique())))


In [18]:
uni(ratings_raw)


 All Unique Value in User-ID
[     2      7      8 ... 278851 278852 278854]
Total no of unique values 105283

 All Unique Value in ISBN
[' 0330299891' ' 0375404120' ' 0586045007' ... '§423350229' '´3499128624'
 'Ô½crosoft']
Total no of unique values 340556

 All Unique Value in Book-Rating
[ 0  1  2  3  4  5  6  7  8  9 10]
Total no of unique values 11


In [19]:
uni(users_raw)


 All Unique Value in User-ID
[     1      2      3 ... 278856 278857 278858]
Total no of unique values 278858

 All Unique Value in Location
['&#19978;&#28023;, shanghai, china'
 '&#19978;&#28023;/&#21488;&#24030;/&#36335;&#26725;, alberta, china'
 '&#20013;&#22269;&#24191;&#19996;&#30465;&#24265;&#27743;&#24066;, florida, china'
 ... 'ýzmir, connecticut, turkey' 'ýzmir, n/a, turkey'
 'ýzmýr, n/a, turkey']
Total no of unique values 57339

 All Unique Value in Age
[  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.
  14.  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.
  28.  29.  30.  31.  32.  33.  34.  35.  36.  37.  38.  39.  40.  41.
  42.  43.  44.  45.  46.  47.  48.  49.  50.  51.  52.  53.  54.  55.
  56.  57.  58.  59.  60.  61.  62.  63.  64.  65.  66.  67.  68.  69.
  70.  71.  72.  73.  74.  75.  76.  77.  78.  79.  80.  81.  82.  83.
  84.  85.  86.  87.  88.  89.  90.  91.  92.  93.  94.  95.  96.  97.
  98.  99. 100. 101. 102. 103. 104

### Checking for Duplicated Values

In [20]:
book_raw[book_raw.duplicated()]

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L


In [21]:
ratings_raw[ratings_raw.duplicated()]

,User-ID,ISBN,Book-Rating


In [22]:
users_raw[users_raw.duplicated()]

,User-ID,Location,Age


- There is no duplicate vale in are dataset

### Visualizing The Data

In [145]:
loc = []

for i in range(len(users_raw['Location'])):

    msg = re.sub('[^a-zA-Z,]',' ',users_raw['Location'][i])
    loc.append(msg)

In [158]:
location = []

for i in range(len(loc)):
    txt = loc[i].split(',')
    location.append(txt)

place = pd.DataFrame(location, columns=['a','b','c','d','e','f','g','h','i'])

place

,a,b,c,d,e,f,g,h,i
0,nyc,new york,usa,None,None,None,None,None,None
1,stockton,california,usa,None,None,None,None,None,None
2,moscow,yukon territory,russia,None,None,None,None,None,None
3,porto,v n gaia,portugal,None,None,None,None,None,None
4,farnborough,hants,united kingdom,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...
278853,portland,oregon,usa,None,None,None,None,None,None
278854,tacoma,washington,united kingdom,None,None,None,None,None,None
278855,brampton,ontario,canada,None,None,None,None,None,None
278856,knoxville,tennessee,usa,None,None,None,None,None,None


In [160]:
user_loc = pd.DataFrame({
    'User-ID' : users_raw['User-ID'],
    'area' : place['a'],
    'city' : place['b'],
    'country' : place['c']
}
)

user_loc

,User-ID,area,city,country
0,1,nyc,new york,usa
1,2,stockton,california,usa
2,3,moscow,yukon territory,russia
3,4,porto,v n gaia,portugal
4,5,farnborough,hants,united kingdom
...,...,...,...,...
278853,278854,portland,oregon,usa
278854,278855,tacoma,washington,united kingdom
278855,278856,brampton,ontario,canada
278856,278857,knoxville,tennessee,usa


In [170]:
user_loc[user_loc['country'].isna()]

,User-ID,area,city,country
29419,29420,st louis,missouri,None
134376,134377,lawrenceville,None,None


In [172]:
user_loc['country'].fillna(user_loc['area'], inplace=True)

In [173]:
user_loc[user_loc['country'].isna()]

,User-ID,area,city,country


In [174]:
user_loc['country'].value_counts().head(10)

 usa               139187
 canada             21558
 united kingdom     18286
 germany            17022
 spain              13095
 australia          11723
 italy              11242
                     4598
 france              3442
 portugal            3306
Name: country, dtype: int64

In [187]:
a = user_loc[user_loc['country'] == 'usa']

a['city'].value_counts(10)

 california               0.071356
 n a                      0.045016
 england                  0.038339
 ontario                  0.031328
 texas                    0.030094
                            ...   
 montgat                  0.000004
 heilongjiang province    0.000004
 klong tuey               0.000004
 silvassa                 0.000004
 denbighshire county      0.000004
Name: city, Length: 6074, dtype: float64

In [180]:
user_loc['country'] = 'usa'